# KNN text classification from ICLR 2018 reviews

Using ICLR 2018 reviews from openreview, KNN was used in this notebook, using the decision as label for a review

In [ ]:
import pandas as pd
import ast # Library to parse code, since replies were saved as a list format in the csv
from collections import Counter
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn import metrics
from matplotlib import pyplot as plt
import string
import re
import numpy as np

from collections import Counter

%matplotlib inline

#### Path to data that contains csv names for iclr conferences

In [ ]:
path_to_data = "iclr_urls.csv"
iclr_conf_data = pd.read_csv(path_to_data) 
iclr_conf_data.head()

#### Showing conference where data is available from the iclr_ulrs.csv

In [ ]:
iclr_conf_data['conference']

#### Data for iclr 2018 is divided between the conference and its workshops

In [ ]:
iclr_conf_data['conference'].iloc[6]

#### Using ICLR 2018 conference data 

In [ ]:
iclr18_dataset = pd.read_csv(iclr_conf_data['conference'].iloc[6]+".csv").dropna()
iclr18_dataset.head()

#### 911 papers are avaible from the openreview website

In [ ]:
replies = iclr18_dataset['replies']
print("Number of papers: " + str(len(replies))) 
replies[:5]

#### There are only 4 possible classes to tag reviews,classes differ a lot in its numbers

In [ ]:
decision_options = Counter(iclr18_dataset['decision'])
decision_options

In [ ]:
decision_labels = {"Accept (Oral)": 0,
         "Invite to Workshop Track": 1,
         "Reject": 2,
         "Accept (Poster)": 3}
decision_labels

In [ ]:
def match_reviews_and_decision(df_venue, labels):
  """Creates and lists of lists containing a review an its decision
  
  Args:
    df_venue (pandas dataframe): Contains data from a venue where each row represents
    a paper that have title, authors, emails, decision, abstract, pdf and replies fields.
    labels (dictionary): Maps a string to a number that encodes the  decision as an integer
  Returns:
    A lists of pairs where the first elements are the decision encoded with the labels
    and the second elements is the review.
    
  """

#   print(df_venue.head())
  
  decision_review = []
  for index, row in df_venue.iterrows():
    comments = ast.literal_eval(row['replies'])
    for comment in comments:
      if comment[1][0] == "rating": # They called reviews as rating
        decision_review.append([labels[row['decision']],comment[1][1]])
  return decision_review

In [ ]:
decision_review = match_reviews_and_decision(iclr18_dataset, decision_labels)
len(decision_review)

#### Defining cleaning method includes punctuation marks, stop words &amp; digits.

In [ ]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    processed = re.sub(r"\d+","",normalized)
    y = processed.split()
    return y

#### Cleaning the reviews and extracting tf-idf features using scikit-learn library

In [ ]:
def preprocess_data(df_venue):
  """Does the preprocessing applying the clean function and the
  TfidVectorizer using the english stopwords
  
  Args:
    df_venue (pandas dataframe): Contains data from a venue where each row represents
    a paper that have title, authors, emails, decision, abstract, pdf and replies fields.
    
  Returns:
    X (sparse matrix, [n_samples, n_features]): Tf-idf-weighted document-term matrix.
    y (lists of integers): labels encoded as integers.

  """
  
  train_clean_sentences = []
  y = np.array([y[0] for y in decision_review])

  for line in decision_review:
    line = line[1].strip()
    cleaned = clean(line)
    cleaned = ' '.join(cleaned)
    train_clean_sentences.append(cleaned)
    
  vectorizer = TfidfVectorizer(stop_words='english')
  
  X = vectorizer.fit_transform(train_clean_sentences)
  
  return X, y

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    y_true = [int(x) for x in y_true]
    y_pred = [int(x) for x in y_pred]
    
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
X, y = preprocess_data(decision_review)

In [ ]:
X.shape

In [ ]:
len(y)

#### Setting k to 4 and printing the confusion matrix for 10 k folds

In [ ]:
modelknn = KNeighborsClassifier(n_neighbors=4)

scores = []
cv = KFold(n_splits=8, random_state=420, shuffle=True)
class_names = np.array(["AcceptOral", "invitedWorkshop", "Reject","AcceptPoster"])

for train_index, test_index in cv.split(X):
  X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
  modelknn.fit(X_train,y_train)
  scores.append(modelknn.score(X_test, y_test))    
  np.set_printoptions(precision=2)
  y_pred = modelknn.fit(X_train, y_train).predict(X_test)

  # Plot non-normalized confusion matrix
  plot_confusion_matrix(y_test, y_pred, classes=class_names, title="Confusion matrix, without normalization")
  # Plot normalized confusion matrix
  plot_confusion_matrix(y_test, y_pred, classes=class_names, normalize=True, title="Normalized confusion matrix")
  plt.show()

In [ ]:
scores

In [ ]:
# Perform 10-fold cross validation
scores = cross_val_score(modelknn, X, y, cv=10)
print("Cross-validated scores:", scores)
print("Avg score: ", sum(scores)/len(scores))